# SAOImage ds9 python lab

Jupyter notebook sandbox for developing new ds9 interaction tools using `samp` to replace the obsolete
`psyds9` module.

Started: 2025 Feb 4

Updated: 2025 Feb 5 - prototyping elements used in modsView v3, but eventually other ds9-based codes.


In [10]:
import os
import sys
import numpy as np

# better way to handle paths

from pathlib import Path

# astropy coordinates for ra/dec and units handling

from astropy.coordinates import SkyCoord, Angle
import astropy.units as u

# better way to handle runtime config files

import yaml

# other bits we need?


### DS9 class

Start of sketching out a "myds9" module DS9 class to encapsulate all the nasty bits of taking to ds9
using samp.  They set/get syntax is very involved even with the convenience functions provided by
the `SAMPIntegratedClient()` functions, and we need to pass info around so a class with internal properties
to hold setup info makes sense.

In [11]:
# astropy has a samp client/hub interaction module

from astropy.samp import SAMPIntegratedClient, SAMPHubError
import time
import subprocess
import shlex




IndentationError: expected an indented block after class definition on line 9 (2221704653.py, line 10)

## Startup process


Instantiate a `SAMPIntegratedClient()` class and try to connect.

If a samp hub is running `connect()` will succeed and set the `is_connected` attribute True.  If we
are connected, then try to find the named ds9 instance `ds9ID` (e.g., `modsView`).  If we don't find
it, start one and then verify we see it in the samp client table.

If no samp hub is running it raises a `SAMPHubError` exception. The corrective is to launch a named ds9
window with the `ds9ID` with the `-samp` flag so that it also starts a samp hub.  This is a nice feature of ds9
that it takes care of launcing a hub for us if needed, or attaches to the running samp hub.

This makes the startup logic complicated as seen below.


In [ ]:
ds9ID = "modsView"
ds9 = DS9(ds9ID)

if ds9.connected:
    if ds9.haveDS9:
        print(f"Connected named ds9 instance {ds9.ds9ID} as samp client {ds9.clientID}")
    else:
        print(f"Connected to a samp hub but no named ds9 instance {ds9.ds9ID} found")
else:
    print(f"could not connect or start a samp hub or ds9 instance")
    

### send ds9 set commands

We got this far we have a ds9 instance to talk to through a samp hub, now configure the ds9 window to our
liking.
 * dimensions: 800x800
 * clear all frames
 * remove redundant pixel output
 * remove colorbar


In [ ]:
ds9.set("width 800")
ds9.set("height 800")
ds9.set("frame clear all")
ds9.set("view image no")        # redundant pixel readout
ds9.set("view colorbar no")     # don't need color bar
ds9.set("raise")

# display an image

fitsFile = "/Users/rwpogge/DEMONEXT/Data/n20190621.GuideCal.i.0006.fits"
ds9.set(f"file fits {fitsFile}")
ds9.set("zoom to fit")
ds9.set("scale mode zscale")

### send ds9 get commands

See if see can get stuff from ds9

In [ ]:
foo = ds9.get("file")
print(foo)

foo = ds9.get("fits")
print(foo)

### iexam test

DS9 remote cursor command is `iexam` 

In [ ]:
#print(f"Put cursor on the {ds9.ds9ID} display and hit any key to return")
#
#cursData = ds9.get("iexam key coordinate image")
#
#cursBits = cursData.split(' ')
#key = cursBits[0]
#cursX = float(cursBits[1])
#cursY = float(cursBits[2])

cursKey, cursX, cursY = ds9.getCursKey()

print(f"Got key={cursKey}, X,Y={cursX:.2f},{cursY:.2f}")
    